In [78]:
from datetime import datetime, timezone

import numpy as np
import pandas as pd

from utils import get_api, get_users_from_user_ids

In [ ]:
# Get my followers (not necessary for now)

In [136]:
# Get all followers ids from target big account
api = get_api()
screen_name = 'MrCluni'
follower_ids = iterate_over_cursors(function=api.get_follower_ids, screen_name=screen_name)
print(len(follower_ids))

19126


In [ ]:
# Get a subset of followers ids from target big account
api = get_api()
screen_name = 'MrCluni'
follower_ids = api.get_follower_ids(screen_name=screen_name)
print(len(follower_ids))

In [5]:
num_users = 100
followers = get_users_from_user_ids(follower_ids[:num_users])

In [143]:
# Make df of followers in account
important_attributes = ['screen_name', 'location', 'followers_count', 'friends_count', 'created_at', 'favourites_count', 'statuses_count']
data = {k:[] for k in important_attributes}
data['last_status'] = []
for user in followers:
    attributes = user._json
    for attribute in important_attributes:
        data[attribute].append(attributes[attribute])
    last_status = np.nan
    if 'status' in attributes:
        last_status = user.status.created_at
    
    data['last_status'].append(last_status)

followers_df = pd.DataFrame(data)
followers_df['friends_followers_ratio'] = followers_df['friends_count'] / followers_df['followers_count']
followers_df['time_processed'] = datetime.now(timezone.utc)
followers_df['days_since_last_status'] = (followers_df['time_processed'] - followers_df['last_status']).dt.days
followers_df['source_screen_name'] = screen_name


In [160]:
# Get users I already checked
df_users_played = pd.read_csv('./data/users_played.csv')

In [156]:
# Pick users to check
num_users = 1
friends_followers_ratio_threshold = 4
days_since_last_status_threshold = 0
friends_threshold = 30

friends_followers_ratio_mask = followers_df['friends_followers_ratio'] > friends_followers_ratio_threshold
days_since_last_status_mask = followers_df['days_since_last_status'] <= days_since_last_status_threshold
friends_mask = followers_df['friends_count'] > friends_threshold
not_played_mask = ~followers_df['screen_name'].isin(df_users_played['screen_name'])

df_to_follow = followers_df[friends_followers_ratio_mask & days_since_last_status_mask & friends_mask & not_played_mask].sample(num_users)

In [157]:
# View users to play
df_to_follow

,screen_name,location,followers_count,friends_count,created_at,favourites_count,statuses_count,last_status,friends_followers_ratio,time_processed,days_since_last_status,source_screen_name
5170,VegetaLiberal,,215,4739,Thu May 06 01:51:06 +0000 2021,3,3730,2021-11-23 20:42:15+00:00,22.04186,2021-11-24 00:19:06.168127+00:00,0.0,MrCluni


In [158]:
# Save users played
df_to_follow.to_csv('./data/users_played.csv', mode='a', index=False)